# Generalizing Euler’s Formula #

This notebook will discuss a well-known formula relating the numbers of faces, vertices and edges in three-dimensional polyhedra, and how to generalize that to account for certain kinds of topological characteristics.

For a much more detailed exposition, which goes into a lot of the concepts of topology, it is worth reading chapter 2 of [Sven Havemann’s PhD dissertation](https://publikationsserver.tu-braunschweig.de/receive/dbbs_mods_00000008).

In order to be able to run the cell content, you will need some extra software installed:
* my [ipy_magics](https://github.com/ldo/ipy_magics), specifically `rman_magic.py`. This provides the ability to render 3D graphics through [Aqsis](http://www.aqsis.org/) and display the results in the notebook.
* [Qahirah](https://github.com/ldo/qahirah), my Python binding for the [Cairo](http://cairographics.org/) 2D graphics library.

In [ ]:
# Edit this as appropriate for the correct path to the Python module
%run ../ipy_magics/rman_magic.py

In [ ]:
import math
deg = math.pi / 180
hypot = lambda *v : math.sqrt(sum(x * x for x in v))

import qahirah as qah
from qahirah import \
    CAIRO, \
    Colour

from IPython.display import \
    display_png

If the above cells execute without errors, then you should have everything properly installed. Following are some common definitions that will be used for rendering meshes and showing information about them. Meshes will be shown in a special way, with little balls marking each vertex, rods defining the edges, and translucent faces allowing you to see through to the other side. This should make the topology of each shape clearer.

In [ ]:
vert_colour = Colour.from_hsva((0.888, 0.75, 0.75))
edge_colour = Colour.from_hsva((0.515, 1.0, 0.76))
face_colour = Colour.from_hsva((0.157, 0.202, 0.89, 0.5))

special_vert_colour = Colour.from_hsva((0.13, 0.987, 0.75))
special_edge_colour = Colour.from_hsva((0.155, 1.0, 0.76))
special_face_colour = Colour.from_hsva((0, 1, 1, 0.75))

def set_colour(c) :
    "given a Colour object, returns the appropriate RenderMan" \
    " command string to set that as the current colour."
    return \
        "Color [%s]" % " ".join("%.3f" % p for p in c.to_rgba()[:3])
#end set_colour

def set_opacity(a) :
    return \
        "Opacity [%s]" % " ".join(["%.3f" % a] * 3)
#end set_opacity

def render_poly(poly) :
    "returns a list of RenderMan statements to render the polyhedron" \
    " defined by poly, which must be a dict with “verts”, “edges” and" \
    " “faces” entries."

    def vertex(pos) :
        return \
          (
            "TransformBegin",
            "Translate %.3g %.3g %.3g" % (pos[0], pos[1], pos[2]),
            "Scale 0.1 0.1 0.1",
            "Sphere 1 -1 1 360",
            "TransformEnd",
          )
    #end vertex

    def edge(pos1, pos2) :
        align_pos_y = math.atan2(pos2[0] - pos1[0], pos2[1] - pos1[1])
        align_pos_z = math.atan2(hypot(pos2[0] - pos1[0], pos2[1] - pos1[1]), pos2[2] - pos1[2])
        return \
          (
            "TransformBegin",
            "Translate %.3g %.3g %.3g" % (pos1[0], pos1[1], pos1[2]),
            "Rotate %.3g %.3g %.3g %.3g" % (- align_pos_y / deg, 0, 0, 1),
            "Rotate %.3g %.3g %.3g %.3g" % (- align_pos_z / deg, 1, 0, 0),
            "Scale 1 1 %.3g" % hypot(pos2[0] - pos1[0], pos2[1] - pos1[1], pos2[2] - pos1[2]),
            "Cylinder %.3g %.3g %.3g %.3g" % (0.05, 0, 1, 360),
            "TransformEnd",
          )
    #end edge

    def face(verts, multi_boundary) :
        result = []
        if multi_boundary :
            runs = []
            coords = []
            for boundary in verts :
                runs.append(len(boundary))
                coords.extend(x for pos in boundary for x in pos)
            #end for
            result.append \
              (
                    "GeneralPolygon [%s] \"P\" [%s]"
                %
                    (
                        " ".join("%d" % n for n in runs),
                        " ".join("%.3g" % x for x in coords),
                    )
              )
        else :
            result.append \
              (
                    "Polygon \"P\" [%s]"
                %
                    " ".join("%.3g" % x for pos in verts for x in pos)
              )
        #end if
        return \
            result
    #end face

#begin render_poly
    result = []
    verts = poly["verts"]
    result.extend \
      ([
        "AttributeBegin",
      ])
    special_verts = set(poly.get("special_verts", ()))
    for special in (False, True) :
        result.append(set_colour((vert_colour, special_vert_colour)[special]))
        for i, v in enumerate(verts) :
            if (i in special_verts) == special :
                result.extend(vertex(v))
            #end if
        #end for
    #end for
    result.extend \
      ([
        "AttributeEnd",
        "AttributeBegin",
      ])
    special_edges = set(poly.get("special_edges", ()))
    for special in (False, True) :
        result.append(set_colour((edge_colour, special_edge_colour)[special]))
        for i, e in enumerate(poly["edges"]) :
            if (i in special_edges) == special :
                result.extend(edge(verts[e[0]], verts[e[1]]))
            #end if
        #end for
    #end for
    result.extend \
      ([
        "AttributeEnd",
        "AttributeBegin",
      ])
    special_faces = set(poly.get("special_faces", ()))
    for special in (False, True) :
        use_colour = (face_colour, special_face_colour)[special]
        result.append(set_colour(use_colour))
        result.append(set_opacity(use_colour.a))
        for i, f in enumerate(poly["faces"]) :
            if (i in special_faces) == special :
                multi_boundary = isinstance(f[0], (list, tuple))
                if multi_boundary :
                    face_verts = tuple(tuple(verts[v] for v in b) for b in f)
                else :
                    face_verts = tuple(verts[v] for v in f)
                #end if
                result.extend(face(face_verts, multi_boundary))
            #end if
        #end for
    #end for
    result.extend \
      ([
        set_opacity(1), # not restored by AttributeEnd!
        "AttributeEnd",
      ])
    return \
        result
#end render_poly

def render_mesh(poly, curved) :
    face_runs = []
    faces = []
    coords = []
    for v in poly["verts"] :
        coords.extend(v)
    #end for
    for f in poly["faces"] :
        multi_boundary = isinstance(f[0], (list, tuple))
        assert not multi_boundary # can’t handle for now
        face_runs.append(len(f))
        faces.extend(f)
    #end for
    return \
        (
                "%s [%s] [%s] \"P\" [%s]"
            %
                (
                    ("PointsPolygons", "SubdivisionMesh \"catmull-clark\"")[curved],
                    " ".join("%d" % l for l in face_runs),
                    " ".join("%d" % l for l in faces),
                    " ".join("%.3g" % c for c in coords),
                )
        )
#end render_mesh

In [ ]:
def show_poly_info(poly, img_bytes) :
    "poly is the polyhedron definition, and img_bytes is the PNG stream" \
    " showing a rendering of the poly. Displays the image side-by-side with" \
    " some information."

    ctx = None
    left_margin = None
    line_y = None
    line_spacing = None

    def darken(c) :
        return \
            c.replace_hsva(v = 0.4, a = 1)
    #end darken

    def show_line(s) :
        nonlocal line_y
        ctx.move_to((left_margin, line_y))
        ctx.show_text(s)
        line_y += line_spacing
    #end show_line

#begin show_poly_info
    poly_img = qah.ImageSurface.create_from_png_bytes(img_bytes)
    pix = qah.ImageSurface.create \
      (
        format = CAIRO.FORMAT_RGB24,
        dimensions = poly_img.dimensions + qah.Vector(300, 0)
      )
    left_margin = poly_img.dimensions.x + 10
    ctx = qah.Context.create(pix)
    ctx.set_source_colour(qah.Colour.grey(1)).paint()
    ctx.set_source_surface(poly_img, (0, 0)).paint()
    ctx.set_font_size(18)
    line_spacing = ctx.font_extents.height
    line_y = 50
    ctx.set_source_colour(darken(face_colour))
    show_line("F = %d" % len(poly["faces"]))
    ctx.set_source_colour(darken(vert_colour))
    show_line("V = %d" % len(poly["verts"]))
    ctx.set_source_colour(darken(edge_colour))
    show_line("E = %d" % len(poly["edges"]))
    ctx.set_source_colour(qah.Colour.grey(0))
    show_line \
      (
            "F + V - E = %d"
        %
            (len(poly["faces"]) + len(poly["verts"]) - len(poly["edges"]))
      )
    if "genus" in poly :
        show_line("G = %d" % poly["genus"])
        if "nr_shells" not in poly and "nr_holes" not in poly :
            show_line \
              (
                    "F + V - E + 2G = %d"
                %
                    (
                        len(poly["faces"])
                    +
                        len(poly["verts"])
                    -
                        len(poly["edges"])
                    +
                        2 * poly["genus"]
                    )
              )
        #end if
    #end if
    if "nr_shells" in poly :
        show_line("B = %d" % poly["nr_shells"])
        if "nr_holes" not in poly :
            show_line \
              (
                    "F + V - E + 2G - 2B = %d"
                %
                    (
                        len(poly["faces"])
                    +
                        len(poly["verts"])
                    -
                        len(poly["edges"])
                    +
                        2 * poly.get("genus", 0)
                    -
                        2 * poly["nr_shells"]
                    )
              )
        #end if
    #end if
    if "nr_holes" in poly :
        show_line("H = %d" % poly["nr_holes"])
        show_line \
          (
                "F + V - E + 2G - 2B - H = %d"
            %
                (
                    len(poly["faces"])
                +
                    len(poly["verts"])
                -
                    len(poly["edges"])
                +
                    2 * poly.get("genus", 0)
                -
                    2 * poly.get("nr_shells", 1)
                -
                    poly["nr_holes"]
                )
          )
    #end if
    display_png(pix.flush().to_png_bytes(), raw = True)
#end show_poly_info


A polyhedron, or mesh, is made up of vertices, edges, and faces. Vertices are points in 3D space; edges connect pairs of vertices; and faces are planar regions bounded by edges.

A *manifold* mesh is one that can define the surface of a real physical object. Specifically, it obeys some restrictions on the connectivity of its parts. These restrictions can be expressed in either of two ways:

The edges and faces that share each vertex can be arranged in a wraparound sequence of alternating faces and edges, such that
  * two successive edges are part of the face between them (and share no  other face), and
  * two successive faces share the edge between them (and share no other edges).
  
Or alternatively:

The edges and vertices that share each face can be arranged in one or more wraparound sequences of alternating vertices and edges, such that, in each sequence
  * two successive edges share the vertex between them (and share no other vertex), and
  * two successive vertices share the edge between then (and share no other edges).

One consequence of these restrictions is that each edge is shared between exactly two faces.

Also the mesh must be *orientable*: that is, you can assign a consistent ordering of the edges for each face, such that the two faces that share an edge include it in their traversal in opposite directions.

Here is an example of a manifold mesh:

In [ ]:
cube = \
    {
        "verts" :
            [
                (-1, -1, -1),
                (-1, -1, 1),
                (-1, 1, -1),
                (-1, 1, 1),
                (1, -1, -1),
                (1, -1, 1),
                (1, 1, -1),
                (1, 1, 1),
            ],

        "edges" :
            [

                (0, 1),
                (2, 3),
                (6, 7),
                (4, 5),

                (0, 2),
                (1, 3),
                (5, 7),
                (4, 6),

                (0, 4),
                (1, 5),
                (3, 7),
                (2, 6),
            ],

        "faces" :
            [
                (0, 1, 3, 2),
                (4, 5, 7, 6),
                (0, 2, 6, 4),
                (1, 3, 7, 5),
                (0, 1, 5, 4),
                (2, 3, 7, 6),
            ],
    }


In [ ]:
%%rman
%rib
%autodisplay

Format 400 400 1

Projection "perspective" "fov" [45]
Translate 0 0 5
LightSource "distantlight" 1

WorldBegin
Rotate 30 1 0 0
%insval "render_poly(cube)"
WorldEnd

And here are a few examples of *non*-manifold meshes. The parts of the mesh that violate the manifold constraints are highlighted in distinguishing colours:

In [ ]:
cube_missing_face = \
  {
    "verts" :
        [
            (1, 1, -1),
            (1, -1, -1),
            (-1, -1, -1),
            (-1, 1, -1),
            (1, 1, 1),
            (1, -1, 1),
            (-1, -1, 1),
            (-1, 1, 1),
        ],
    "special_verts" : [4, 5, 6, 7],
    "edges" :
        [
            (0, 1),
            (0, 3),
            (0, 4),
            (1, 2),
            (1, 5),
            (2, 3),
            (2, 6),
            (3, 7),
            (4, 5),
            (4, 7),
            (5, 6),
            (6, 7),
        ],
    "special_edges" : [8, 9, 10, 11],
    "faces" :
        [
            (0, 1, 2, 3),
            (0, 4, 5, 1),
            (1, 5, 6, 2),
            (2, 6, 7, 3),
            (4, 0, 3, 7),
        ],
  }


In [ ]:
%%rman
%rib
%autodisplay

Format 400 400 1

Projection "perspective" "fov" [45]
Translate 0 0 5
LightSource "distantlight" 1

WorldBegin
Rotate -180 1 1 0
Rotate 35 0 1 0
%insval "render_poly(cube_missing_face)"
WorldEnd

In [ ]:
tetra_plus_vert = \
  {
    "verts" :
        [
            (0, 0, 1),
            (0.943, 0, -0.333),
            (-0.471, 0.816, -0.333),
            (-0.471, -0.816, -0.333),
            (0, 0, 2.46),
        ],
    "special_verts" : [0, 4],
    "edges" :
        [
            (0, 3),
            (1, 3),
            (2, 3),
            (1, 2),
            (0, 1),
            (0, 2),
            (0, 4),
        ],
    "special_edges" : [6],
    "faces" :
        [
            (0, 1, 2),
            (0, 2, 3),
            (0, 3, 1),
            (1, 3, 2),
        ],
  }


In [ ]:
%%rman
%rib
%autodisplay

Format 400 400 1

Projection "perspective" "fov" [45]
Translate 0.5 -0.5 4
LightSource "distantlight" 1

WorldBegin
Rotate -90 1 1 0
%insval "render_poly(tetra_plus_vert)"
WorldEnd

In [ ]:
cube_plus_interior_face = \
  {
    "verts" :
        [
            (-1, -1, -1),
            (-1, -1, 1),
            (-1, 1, -1),
            (-1, 1, 1),
            (1, -1, -1),
            (1, -1, 1),
            (1, 1, -1),
            (1, 1, 1),
            (-0.048, -0.0315, -0.765),
            (-0.048, -0.0315, 0.765),
        ],
    "special_verts" : [6, 7, 8, 9],
    "edges" :
        [
            (2, 0),
            (0, 1),
            (1, 3),
            (3, 2),
            (6, 2),
            (3, 7),
            (7, 6),
            (4, 6),
            (7, 5),
            (5, 4),
            (0, 4),
            (5, 1),
            (9, 8),
            (6, 8),
            (9, 7),
        ],
    "special_edges" : [6, 12, 13, 14],
    "faces" :
        [
            (0, 1, 3, 2),
            (2, 3, 7, 6),
            (6, 7, 5, 4),
            (4, 5, 1, 0),
            (2, 6, 4, 0),
            (7, 3, 1, 5),
            (7, 6, 8, 9),
        ],
    "special_faces" : [6],
  }


In [ ]:
%%rman
%rib
%autodisplay

Format 400 400 1

Projection "perspective" "fov" [45]
Translate 0 0 5
LightSource "distantlight" 1

WorldBegin
Rotate -40 1 1 0
Rotate 135 0 0 1
%insval "render_poly(cube_plus_interior_face)"
WorldEnd

In [ ]:
shared_vertex = \
  {
    "verts" :
        [
            (0, 0, 1),
            (0.943, 0, -0.333),
            (-0.471, 0.816, -0.333),
            (-0.471, -0.816, -0.333),
            (0, 1, 2.23),
            (1, -4.37e-08, 2.23),
            (-8.74e-08, -1, 2.23),
            (-1, 1.19e-08, 2.23),
        ],
    "special_verts" : [0],
    "edges" :
        [
            (0, 3),
            (1, 3),
            (2, 3),
            (1, 2),
            (0, 1),
            (0, 2),
            (4, 5),
            (5, 6),
            (6, 7),
            (7, 4),
            (0, 6),
            (0, 5),
            (4, 0),
            (0, 7),
        ],
    "faces" :
        [
            (0, 1, 2),
            (0, 2, 3),
            (0, 3, 1),
            (1, 3, 2),
            (4, 0, 5),
            (5, 0, 6),
            (6, 0, 7),
            (7, 0, 4),
            (4, 5, 6, 7),
        ],
  }


In [ ]:
%%rman
%rib
%autodisplay

Format 400 400 1

Projection "perspective" "fov" [45]
Translate -0.5 0.5 4
LightSource "distantlight" 1

WorldBegin
Rotate 60 1 1 0
%insval "render_poly(shared_vertex)"
WorldEnd

In the following, we will start by considering manifold polyhedra that are *topologically equivalent to a sphere*. But what does that mean, exactly?

Consider a sphere, and consider a closed loop drawn on its surface. No matter how you draw this loop, it will *partition* the surface into two parts, such that, to get from a surface point in one partition to one in the other while staying on the surface, you have to cross the loop boundary. The same thing happens on the surface of any mesh that is topologically equivalent to a sphere.

In the following example, the two partitions of the surface are drawn in contrasting colours.

In [ ]:
%%rman
%rib
%autodisplay

Format 300 300 1

Projection "perspective" "fov" [45]
Translate 0 0 3

LightSource "distantlight" 1 "intensity" [0.5] "to" [-1 -1 0.5]
LightSource "ambientlight" 2 "intensity" [0.4]

WorldBegin
Rotate 150 1 1 0
TransformBegin
Scale 1 1 1
Surface "plastic" "roughness" [0.9]
%insval "set_colour(Colour.from_hsva((0.25, 0.7, 0.9)))"
Sphere 1 -1 0.9 360
%insval "set_colour(Colour.from_hsva((0.15, 0.7, 0.7)))"
Sphere 1 0.9 1 360
TransformEnd
WorldEnd

The maximum number of closed loops that can be drawn on a surface *without* partitioning the surface into separate regions in this way is denoted as $B_1$, one of a series of *Betti numbers* $B_i$. Thus, for the sphere, $B_1 = 0$.

It seems that $B_1$ is always even. Thus, the *genus* of a shape $G = {B_1 \over 2}$, and for the sphere, $G = 0$.

For a torus, it is possible to draw up to *two* such closed loops *without* partitioning the surface, if you position them correctly.

In [ ]:
nurbs_circle_8 = \
    [
        [
            (0, -1, 0),
            (-0.445, -0.89, 0),
            (-0.71, -0.71, 0),
            (-0.89, -0.445, 0),
            (-1, 0, 0),
            (-0.89, 0.445, 0),
            (-0.71, 0.71, 0),
            (-0.445, 0.89, 0),
            (0, 1, 0),
            (0.445, 0.89, 0),
            (0.71, 0.71, 0),
            (0.89, 0.445, 0),
            (1, 0, 0),
            (0.89, -0.445, 0),
            (0.71, -0.71, 0),
            (0.445, -0.89, 0),
        ],
    ]

def render_curve(c, cubic, periodic, width) :
    curves = []
    coords = []
    for seg in c :
        curves.append(len(seg))
        for pt in seg :
            coords.extend(pt)
        #end for
    #end for
    return \
      (
            "Curves \"%s\" [%s] \"%s\" \"P\" [%s] \"constantwidth\" %.3g"
        %
            (
                ("linear", "cubic")[cubic],
                " ".join("%d" % l for l in curves),
                ("nonperiodic", "periodic")[periodic],
                " ".join("%.3g" % c for c in coords),
                width,
            )
      )
#end render_curve

In [ ]:
%%rman
%rib
%autodisplay

Format 300 300 1

Projection "perspective" "fov" [45]
Translate 0 0 11

LightSource "distantlight" 1 "intensity" [0.5] "to" [-1 -1 0.5]
LightSource "ambientlight" 2 "intensity" [0.4]

WorldBegin
Rotate 45 0 1 0
TransformBegin
Scale 1 1 1
Surface "plastic" "roughness" [0.9]
%insval "set_colour(Colour.from_hsva((0.25, 0.7, 0.9)))"
Torus 3 1 0 360 360
%insval "set_colour(Colour.grey(0.3))"
TransformBegin
Translate 3 0 0
Rotate 90 1 0 0
Scale 1.05 1.05 1.05
Basis "b-spline" 1 "b-spline" 1
%insval "render_curve(nurbs_circle_8, True, True, 0.05)"
TransformEnd
TransformBegin
TransformEnd
Scale 3 3 3
Translate 0 0 -0.35
Basis "b-spline" 1 "b-spline" 1
%insval "render_curve(nurbs_circle_8, True, True, 0.02)"
TransformEnd
WorldEnd

As you can see, it is still possible to reach every point on the green part of the surface from every other point while staying on the surface, without crossing the dark lines. If you add a third loop, no matter where, this will no longer be possible.

Thus, for the torus, $B_1 = 2$, therefore $G = 1$. We will come back to shapes with nonzero values of $G$ later.

For the case of a manifold polyhedron that is topologically equivalent to a sphere ($G = 0$), where $V$ is the number of vertices, $E$ is the number of edges and $F$ the number of faces, we have the original form of Euler’s formula:

$$F + V = E + 2$$

Some examples:

In [ ]:
%%rman --return-var=cube_img --post-process="show_poly_info(cube, cube_img[0])"
%rib
%autodisplay

Format 400 400 1

Projection "perspective" "fov" [45]
Translate 0 0 5
LightSource "distantlight" 1

WorldBegin
Rotate 30 1 1 0
%insval "render_poly(cube)"
WorldEnd

In [ ]:
dodecahedron = \
  {
    "verts" :
        [
            (0.866, 0.866, 0.866),
            (0.866, 0.866, -0.866),
            (0.866, -0.866, 0.866),
            (0.866, -0.866, -0.866),
            (-0.866, 0.866, 0.866),
            (-0.866, 0.866, -0.866),
            (-0.866, -0.866, 0.866),
            (-0.866, -0.866, -0.866),
            (0.535, 1.4, 0),
            (-0.535, 1.4, 0),
            (0.535, -1.4, 0),
            (-0.535, -1.4, 0),
            (1.4, 0, 0.535),
            (1.4, 0, -0.535),
            (-1.4, 0, 0.535),
            (-1.4, 0, -0.535),
            (0, 0.535, 1.4),
            (0, -0.535, 1.4),
            (0, 0.535, -1.4),
            (0, -0.535, -1.4),
        ],
    "edges" :
        [
            (8, 9),
            (6, 14),
            (4, 9),
            (16, 17),
            (0, 16),
            (6, 17),
            (7, 15),
            (7, 19),
            (10, 11),
            (4, 16),
            (6, 11),
            (2, 17),
            (1, 13),
            (18, 19),
            (3, 19),
            (3, 10),
            (5, 18),
            (12, 13),
            (2, 12),
            (7, 11),
            (0, 8),
            (2, 10),
            (1, 18),
            (14, 15),
            (5, 15),
            (4, 14),
            (3, 13),
            (5, 9),
            (1, 8),
            (0, 12),
        ],
    "faces" :
        [
            (4, 14, 6, 17, 16),
            (19, 18, 1, 13, 3),
            (8, 9, 4, 16, 0),
            (12, 13, 1, 8, 0),
            (11, 6, 14, 15, 7),
            (19, 3, 10, 11, 7),
            (16, 17, 2, 12, 0),
            (1, 18, 5, 9, 8),
            (5, 15, 14, 4, 9),
            (15, 5, 18, 19, 7),
            (11, 10, 2, 17, 6),
            (2, 10, 3, 13, 12),
        ],
  }


In [ ]:
%%rman --return-var=dodeca_img --post-process="show_poly_info(dodecahedron, dodeca_img[0])"
%rib
%autodisplay

Format 400 400 1

Projection "perspective" "fov" [45]
Translate 0 0 5
LightSource "distantlight" 1

WorldBegin
Rotate 25 0 0 1
Rotate 5 0 1 0
%insval "render_poly(dodecahedron)"
WorldEnd

In [ ]:
triangular_prism = \
  {
    "verts" :
        [
            (0, 1, -1),
            (0, 1, 1),
            (0.866, -0.5, -1),
            (0.866, -0.5, 1),
            (-0.866, -0.5, -1),
            (-0.866, -0.5, 1),
        ],
    "edges" :
        [
            (0, 2),
            (3, 1),
            (0, 1),
            (3, 2),
            (2, 4),
            (5, 3),
            (5, 4),
            (4, 0),
            (1, 5),
        ],
    "faces" :
        [
            (0, 1, 3, 2),
            (2, 3, 5, 4),
            (3, 1, 5),
            (4, 5, 1, 0),
            (0, 2, 4),
        ],
  }


In [ ]:
%%rman --return-var=triprism_img --post-process="show_poly_info(triangular_prism, triprism_img[0])"
%rib
%autodisplay

Format 400 400 1

Projection "perspective" "fov" [35]
Translate 0 0 5
LightSource "distantlight" 1

WorldBegin
Rotate 60 0 1 0
%insval "render_poly(triangular_prism)"
WorldEnd

But what about a shape that is topologically equivalent to a doughnut or a coffee cup?

In [ ]:
mug = \
  {
    "verts" :
        [
            (-1, -1, 0),
            (1, -1, 0),
            (-1, 1, 0),
            (1, 1, 0),
            (-0.635, 0.635, 2.47),
            (-0.635, -0.635, 2.47),
            (0.635, -0.635, 2.47),
            (0.635, 0.635, 2.47),
            (-1, 1, 2.47),
            (-1, -1, 2.47),
            (1, -1, 2.47),
            (1, 1, 2.47),
            (-0.635, 0.635, 0.702),
            (-0.635, -0.635, 0.702),
            (0.635, -0.635, 0.702),
            (0.635, 0.635, 0.702),
            (1, -0.395, 1.49),
            (1, 0.395, 1.49),
            (1, -0.395, 2.16),
            (1, 0.395, 2.16),
            (1, -0.395, 0.442),
            (1, 0.395, 0.442),
            (1, -0.395, 1.11),
            (1, 0.395, 1.11),
            (1.76, -0.395, 1.49),
            (1.76, 0.395, 1.49),
            (1.76, -0.395, 2.16),
            (1.76, 0.395, 2.16),
            (1.76, -0.395, 0.442),
            (1.76, 0.395, 0.442),
            (1.76, -0.395, 1.11),
            (1.76, 0.395, 1.11),
            (2.44, -0.395, 1.49),
            (2.44, 0.395, 1.49),
            (2.44, -0.395, 2.16),
            (2.44, 0.395, 2.16),
            (2.44, -0.395, 0.442),
            (2.44, 0.395, 0.442),
            (2.44, -0.395, 1.11),
            (2.44, 0.395, 1.11),
            (1, 1, 1.23),
            (1, -1, 1.23),
            (-1, -1, 1.23),
            (-1, 1, 1.23),
            (1, -0.189, 1.3),
            (1, 0.189, 1.3),
        ],
    "edges" :
        [
            (2, 0),
            (0, 1),
            (1, 3),
            (3, 2),
            (8, 9),
            (9, 10),
            (10, 11),
            (11, 8),
            (40, 11),
            (41, 1),
            (42, 9),
            (43, 2),
            (4, 5),
            (5, 6),
            (6, 7),
            (7, 4),
            (9, 5),
            (4, 8),
            (10, 6),
            (11, 7),
            (12, 13),
            (13, 14),
            (14, 15),
            (15, 12),
            (4, 12),
            (15, 7),
            (6, 14),
            (13, 5),
            (16, 17),
            (18, 19),
            (17, 19),
            (18, 16),
            (20, 21),
            (22, 23),
            (21, 23),
            (22, 20),
            (24, 25),
            (26, 27),
            (25, 27),
            (26, 24),
            (28, 29),
            (30, 31),
            (29, 31),
            (30, 28),
            (21, 29),
            (28, 20),
            (19, 27),
            (25, 17),
            (24, 16),
            (30, 22),
            (31, 23),
            (26, 18),
            (32, 33),
            (34, 35),
            (33, 35),
            (34, 32),
            (36, 37),
            (38, 39),
            (37, 39),
            (38, 36),
            (28, 36),
            (38, 30),
            (39, 31),
            (24, 32),
            (34, 26),
            (35, 27),
            (37, 29),
            (33, 25),
            (24, 30),
            (38, 32),
            (39, 33),
            (31, 25),
            (3, 40),
            (10, 41),
            (0, 42),
            (8, 43),
            (41, 42),
            (43, 40),
            (42, 43),
            (41, 16),
            (18, 10),
            (19, 11),
            (17, 40),
            (1, 20),
            (3, 21),
            (41, 22),
            (23, 40),
            (44, 16),
            (44, 22),
            (45, 17),
            (45, 23),
            (44, 45),
        ],
    "faces" :
        [
            (0, 2, 3, 1),
            (7, 4, 12, 15),
            (43, 42, 9, 8),
            (40, 43, 8, 11),
            (42, 41, 10, 9),
            (5, 4, 8, 9),
            (6, 5, 9, 10),
            (7, 6, 10, 11),
            (4, 7, 11, 8),
            (13, 14, 15, 12),
            (5, 6, 14, 13),
            (6, 7, 15, 14),
            (4, 5, 13, 12),
            (41, 16, 18, 10),
            (32, 38, 39, 33),
            (30, 28, 36, 38),
            (20, 21, 29, 28),
            (17, 19, 27, 25),
            (16, 17, 25, 24),
            (22, 20, 28, 30),
            (23, 22, 30, 31),
            (18, 16, 24, 26),
            (19, 18, 26, 27),
            (21, 23, 31, 29),
            (32, 33, 35, 34),
            (36, 37, 39, 38),
            (26, 24, 32, 34),
            (27, 26, 34, 35),
            (29, 31, 39, 37),
            (28, 29, 37, 36),
            (25, 27, 35, 33),
            (24, 30, 38, 32),
            (33, 39, 31, 25),
            (25, 31, 30, 24),
            (0, 1, 41, 42),
            (3, 2, 43, 40),
            (2, 0, 42, 43),
            (10, 18, 19, 11),
            (11, 19, 17, 40),
            (3, 21, 20, 1),
            (1, 20, 22, 41),
            (23, 21, 3, 40),
            (44, 16, 41, 22),
            (23, 40, 17, 45),
            (23, 45, 44, 22),
            (17, 16, 44, 45),
        ],
    "genus" : 1,
  }
mug_colour = Colour.from_hsva((0.24, 0.38, 0.68, 0.7))

In [ ]:
%%rman
%rib
%autodisplay

Format 400 500 1

Projection "perspective" "fov" [45]
Translate -0.5 1 7
LightSource "distantlight" 1

WorldBegin
%insval "set_colour(mug_colour)"
%insval "set_opacity(mug_colour.a)"
TransformBegin
Translate 0 -0.5 0
Rotate -135 1 0 0
Rotate 20 0 0 1
%insval "render_mesh(mug, False)"
TransformEnd
Translate 0 -4 0
TransformBegin
Rotate -115 1 0 0
Rotate 20 0 0 1
%insval "render_mesh(mug, True)"
TransformEnd
WorldEnd

It is tempting to call the gap opened up in the shape a “hole”, but in fact that has a specific—and different—meaning that will be introduced later. Instead, the term for what we talking about here is a *handle*—because of course it is equivalent to the handle in a coffee cup.

Following are examples of shapes with one or two handles. The number of handles is also known as the *genus* of the polyhedron.

In [ ]:
rect_torus = \
    {
        "verts" :
            [
                (-1, -1, 0.5),
                (1, -1, 0.5),
                (1, 1, 0.5),
                (-1, 1, 0.5),
                (-0.75, -0.75, 0),
                (0.75, -0.75, 0),
                (0.75, 0.75, 0),
                (-0.75, 0.75, 0),
                (-1.25, -1.25, 0),
                (1.25, -1.25, 0),
                (1.25, 1.25, 0),
                (-1.25, 1.25, 0),
                (-1, -1, -0.5),
                (1, -1, -0.5),
                (1, 1, -0.5),
                (-1, 1, -0.5),
            ],
        "edges" :
            [
                (0, 1),
                (1, 2),
                (2, 3),
                (3, 0),
                (4, 5),
                (5, 6),
                (6, 7),
                (7, 4),
                (8, 9),
                (9, 10),
                (10, 11),
                (11, 8),
                (12, 13),
                (13, 14),
                (14, 15),
                (15, 12),

                (0, 4),
                (4, 12),
                (12, 8),
                (8, 0),
                (1, 5),
                (5, 13),
                (13, 9),
                (9, 1),
                (2, 6),
                (6, 14),
                (14, 10),
                (10, 2),
                (3, 7),
                (7, 15),
                (15, 11),
                (11, 3),
            ],
        "faces" :
            [
                (0, 4, 5, 1),
                (1, 5, 6, 2),
                (2, 6, 7, 3),
                (3, 7, 4, 0),

                (0, 8, 9, 1),
                (1, 9, 10, 2),
                (2, 10, 11, 3),
                (3, 11, 8, 0),

                (12, 8, 9, 13),
                (13, 9, 10, 14),
                (14, 10, 11, 15),
                (15, 11, 8, 12),

                (12, 4, 5, 13),
                (13, 5, 6, 14),
                (14, 6, 7, 15),
                (15, 7, 4, 12),
            ],
        "genus" : 1,
    }


In [ ]:
%%rman --return-var=torus_img --post-process="show_poly_info(rect_torus, torus_img[0])"
%rib
%autodisplay

Format 400 400 1

Projection "perspective" "fov" [45]
Translate 0 0 5
LightSource "distantlight" 1

WorldBegin
Rotate 35 0 1 1
Rotate 20 1 0 0
%insval "render_poly(rect_torus)"
WorldEnd

In [ ]:
%%rman --return-var=mug_img --post-process="show_poly_info(mug, mug_img[0])"
%rib
%autodisplay

Format 400 400 1

Projection "perspective" "fov" [45]
Translate -0.5 -1 7
LightSource "distantlight" 1

WorldBegin
Rotate -115 1 0 0
Rotate 20 0 0 1
%insval "render_poly(mug)"
WorldEnd

In [ ]:
rect_bitorus = \
  {
    "verts" :
        [
            (-1, -1.5, 0),
            (1, -1.5, 0),
            (-1, 0.5, 0),
            (1, 0.5, 0),
            (-1, 1, 0),
            (1, 1, 0),
            (-1, 3, 0),
            (1, 3, 0),
            (-1.5, -2, 0),
            (1.5, -2, 0),
            (1.25, -1.75, 0.25),
            (-1.25, -1.75, 0.25),
            (-1.5, 3.5, 0),
            (1.5, 3.5, 0),
            (-1.25, 0.75, 0.25),
            (1.25, 0.75, 0.25),
            (-1.25, 3.25, 0.25),
            (1.25, 3.25, 0.25),
            (1.25, -1.75, -0.25),
            (-1.25, -1.75, -0.25),
            (-1.25, 0.75, -0.25),
            (1.25, 0.75, -0.25),
            (-1.5, 0.75, 0),
            (-1.25, 3.25, -0.25),
            (1.25, 3.25, -0.25),
            (1.5, 0.75, 0),
        ],
    "edges" :
        [
            (2, 0),
            (0, 1),
            (1, 3),
            (3, 2),
            (6, 4),
            (4, 5),
            (5, 7),
            (7, 6),
            (8, 9),
            (11, 10),
            (3, 15),
            (21, 24),
            (14, 11),
            (13, 12),
            (23, 20),
            (10, 15),
            (16, 14),
            (14, 15),
            (15, 17),
            (17, 16),
            (18, 21),
            (19, 18),
            (20, 19),
            (21, 20),
            (24, 23),
            (8, 11),
            (9, 10),
            (8, 19),
            (9, 18),
            (13, 17),
            (12, 16),
            (13, 24),
            (12, 23),
            (7, 17),
            (6, 16),
            (5, 15),
            (4, 14),
            (0, 11),
            (1, 10),
            (0, 19),
            (1, 18),
            (3, 21),
            (2, 20),
            (7, 24),
            (6, 23),
            (2, 14),
            (4, 20),
            (5, 21),
            (22, 8),
            (22, 20),
            (25, 9),
            (25, 21),
            (15, 25),
            (13, 25),
            (22, 12),
            (22, 14),
        ],
    "faces" :
        [
            (9, 10, 11, 8),
            (9, 8, 19, 18),
            (12, 16, 17, 13),
            (12, 13, 24, 23),
            (6, 7, 17, 16),
            (7, 5, 15, 17),
            (4, 6, 16, 14),
            (5, 4, 14, 15),
            (1, 0, 11, 10),
            (1, 18, 19, 0),
            (3, 1, 10, 15),
            (2, 3, 15, 14),
            (0, 2, 14, 11),
            (3, 21, 18, 1),
            (0, 19, 20, 2),
            (2, 20, 21, 3),
            (5, 21, 20, 4),
            (7, 24, 21, 5),
            (6, 23, 24, 7),
            (4, 20, 23, 6),
            (8, 22, 20, 19),
            (25, 9, 18, 21),
            (10, 9, 25, 15),
            (15, 25, 13, 17),
            (21, 24, 13, 25),
            (16, 12, 22, 14),
            (23, 20, 22, 12),
            (11, 14, 22, 8),
        ],
    "genus" : 2,
  }


In [ ]:
%%rman --return-var=bitorus_img --post-process="show_poly_info(rect_bitorus, bitorus_img[0])"
%rib
%autodisplay

Format 400 400 1

Projection "perspective" "fov" [50]
Translate 0 0 7
LightSource "distantlight" 1

WorldBegin
Rotate 75 0 0 1
Rotate 30 1 0 0
Rotate -25 0 1 0
%insval "render_poly(rect_bitorus)"
WorldEnd

If $G$ is the number of handles, Euler’s formula generalizes to

$$F + V = E + 2(1 - G)$$

or conversely

$$F + V + 2G = E + 2$$

The next step is to allow multiple discontiguous *shells* to make up a single object. That is, a single object is now allowed to be made up of more than one separate polyhedron. In this case, the formula applies individually to each shell: simply total up all the $F$s, $V$s, $E$s and $G$s, and multiply the constant factor 2 by the number of shells.

In [ ]:
two_cubes = \
  {
    "verts" :
        [
            (1, 1, -1),
            (1, -1, -1),
            (-1, -1, -1),
            (-1, 1, -1),
            (1, 1, 1),
            (1, -1, 1),
            (-1, -1, 1),
            (-1, 1, 1),
            (-1.42, -2.43, -0.8),
            (0.0425, -3.47, -1.67),
            (0.0659, -4.73, -0.108),
            (-1.4, -3.68, 0.758),
            (-0.0659, -1.27, 0.108),
            (1.4, -2.32, -0.758),
            (1.42, -3.57, 0.8),
            (-0.0425, -2.53, 1.67),
        ],
    "edges" :
        [
            (0, 1),
            (0, 3),
            (0, 4),
            (1, 2),
            (1, 5),
            (2, 3),
            (2, 6),
            (3, 7),
            (4, 5),
            (4, 7),
            (5, 6),
            (6, 7),
            (8, 9),
            (8, 11),
            (8, 12),
            (9, 10),
            (9, 13),
            (10, 11),
            (10, 14),
            (11, 15),
            (12, 13),
            (12, 15),
            (13, 14),
            (14, 15),
        ],
    "faces" :
        [
            (0, 1, 2, 3),
            (4, 7, 6, 5),
            (0, 4, 5, 1),
            (1, 5, 6, 2),
            (2, 6, 7, 3),
            (4, 0, 3, 7),
            (8, 9, 10, 11),
            (12, 15, 14, 13),
            (8, 12, 13, 9),
            (9, 13, 14, 10),
            (10, 14, 15, 11),
            (12, 8, 11, 15),
        ],
    "genus" : 0,
    "nr_shells" : 2,
  }


In [ ]:
%%rman --return-var=two_cubes_img --post-process="show_poly_info(two_cubes, two_cubes_img[0])"
%rib
%autodisplay

Format 400 400 1

Projection "perspective" "fov" [45]
Translate -1 1 9
LightSource "distantlight" 1

WorldBegin
Rotate 35 0 0 1
%insval "render_poly(two_cubes)"
WorldEnd

Therefore, if the number of shells is $B$, we have in total

$$F + V + 2G = E + 2B$$

or

$$F + V = E + 2(B - G)$$

Finally, let us allow for a nonzero number $H$ of *holes*, that is, rings of discontiguous edge loops in the interior of a face. Perhaps a better name than “hole” would have been “interior boundary”. Every face has an exterior boundary; additional boundaries inside this demarcate the holes.

The following example shows a single face with one hole in it:

In [ ]:
hole_face_example_1 = \
  {
    "verts" :
        [
            (0.476, 0.476, 1),
            (0.476, -0.476, 1),
            (-0.476, -0.476, 1),
            (-0.476, 0.476, 1),
            (1, 1, 1),
            (1, -1, 1),
            (-1, 1, 1),
            (-1, -1, 1),
        ],
    "edges" :
        [
            (4, 5),
            (4, 6),
            (5, 7),
            (7, 6),
            (0, 1),
            (0, 3),
            (1, 2),
            (2, 3),
        ],
    "faces" :
        [
            [(0, 1, 2, 3), (4, 6, 7, 5)],
        ],
  }


In [ ]:
%%rman
%rib
%autodisplay

Format 400 300 1

Projection "perspective" "fov" [45]
Translate 0 0 4
LightSource "distantlight" 1

WorldBegin
Rotate -150 1 0 0
%insval "render_poly(hole_face_example_1)"
WorldEnd

Of course, a face can have more than one hole:

In [ ]:
hole_face_example_2 = \
  {
    "verts" :
        [
            (0, 1, 0),
            (-0.866, 0.5, 0),
            (-0.866, -0.5, 0),
            (8.74e-08, -1, 0),
            (0.866, -0.5, 0),
            (0.866, 0.5, 0),
            (0.421, 0.0863, 3.73e-09),
            (0.161, -0.364, 3.73e-09),
            (0.681, -0.364, 3.73e-09),
            (-0.296, 0.496, 0),
            (-0.582, 0.288, 0),
            (-0.473, -0.0469, 0),
            (-0.12, -0.0469, 0),
            (-0.0111, 0.288, 0),
        ],
    "edges" :
        [
            (1, 0),
            (2, 1),
            (3, 2),
            (4, 3),
            (5, 4),
            (0, 5),
            (7, 6),
            (8, 7),
            (6, 8),
            (10, 9),
            (11, 10),
            (12, 11),
            (13, 12),
            (9, 13),
        ],
    "faces" :
        [
            [(0, 1, 2, 3, 4, 5), (9, 10, 11, 12, 13), (6, 7, 8)]
        ],
  }


In [ ]:
%%rman
%rib
%autodisplay

Format 400 300 1

Projection "perspective" "fov" [45]
Translate 0 0.2 2.5
LightSource "distantlight" 1

WorldBegin
Rotate -150 1 0 0
%insval "render_poly(hole_face_example_2)"
WorldEnd

Next is a complete manifold polyhedron with such a face:

In [ ]:
hole_cube = \
  {
    "verts" :
        [
            (1, 1, -1),
            (1, -1, -1),
            (-1, -1, -1),
            (-1, 1, -1),
            (0.476, 0.476, 1),
            (0.476, -0.476, 1),
            (-0.476, -0.476, 1),
            (-0.476, 0.476, 1),
            (1, 1, 1),
            (1, -1, 1),
            (-1, 1, 1),
            (-1, -1, 1),
            (0.476, 0.476, 2.3),
            (0.476, -0.476, 2.3),
            (-0.476, 0.476, 2.3),
            (-0.476, -0.476, 2.3),
        ],
    "edges" :
        [
            (0, 1),
            (0, 3),
            (0, 8),
            (1, 2),
            (1, 9),
            (2, 3),
            (2, 11),
            (3, 10),
            (8, 9),
            (8, 10),
            (9, 11),
            (11, 10),
            (4, 5),
            (4, 7),
            (5, 6),
            (6, 7),
            (12, 13),
            (12, 14),
            (13, 15),
            (15, 14),
            (5, 13),
            (15, 6),
            (4, 12),
            (14, 7),
        ],
    "faces" :
        [
            (0, 1, 2, 3),
            (6, 5, 13, 15),
            (0, 8, 9, 1),
            (1, 9, 11, 2),
            (2, 11, 10, 3),
            (8, 0, 3, 10),
            (12, 14, 15, 13),
            (5, 4, 12, 13),
            (7, 6, 15, 14),
            (4, 7, 14, 12),
            [(8, 10, 11, 9), (4, 5, 6, 7)],
        ],
    "nr_holes" : 1,
  }


In [ ]:
%%rman --return-var=hole_cube_img --post-process="show_poly_info(hole_cube, hole_cube_img[0])"
%rib
%autodisplay

Format 300 400 1

Projection "perspective" "fov" [45]
Translate -0.3 -0.3 6
LightSource "distantlight" 1

WorldBegin
Rotate -120 1 0 0
%insval "render_poly(hole_cube)"
WorldEnd

This generalization is most easily understood in terms of the possible operations that can add such holes to an object.

One way is by a merging of two shells so they share a face in common, such that the outer boundary of the smaller face becomes a hole in the larger one (the previous illustration could have come about in this way by merging two cubes). In other words, a face and a shell are subtracted, and a hole is added. So the changes in $F$, $B$ and $H$ have to satisfy

$$\Delta F + \Delta B + 2 \Delta H = 0$$

(assuming nothing else changes).

Therefore, in the previous formula, we can try replacing $F$ with $F + H$, $G$ with $F + G$ and $B$ with $B + H$, to give

$$F + H + V = E + 2(H + B - F - G) = E + 2H + 2B - 2F - 2G$$

or

$$F + V = E + H + 2(B - G)$$

or

$$F + V + 2G = E + H + 2B$$

which is known as the *extended Euler-Poincaré equation*.

But there is another way to create a hole, and that is by merging two faces of the same shell, so one ends up as a hole inside the other. This subtracts a face and adds a handle and a hole. But we can fit this into the same formula by saying that the changes in $F$, $G$ and $H$ also have to satisfy

$$\Delta F + 0\Delta G + \Delta H = 0$$

(again assuming nothing else changes).

In [ ]:
cuboboros = \
  {
    "verts" :
        [
            (-0.281, -0.5, -1.6),
            (-0.721, -0.5, -0.697),
            (-0.281, 0.5, -1.6),
            (-0.721, 0.5, -0.697),
            (0.208, -0.5, -1.55),
            (0.789, -0.5, -0.739),
            (0.208, 0.5, -1.55),
            (0.789, 0.5, -0.739),
            (-0.832, 0.5, -1.9),
            (-0.832, -0.5, -1.9),
            (-1.83, -0.5, -1.92),
            (-1.83, 0.5, -1.92),
            (1.59, 0.5, -2.13),
            (0.589, 0.5, -2.06),
            (0.589, -0.5, -2.06),
            (1.59, -0.5, -2.13),
            (-0.386, 0.5, -2.46),
            (-0.386, -0.5, -2.46),
            (-0.929, -0.5, -3.3),
            (-0.929, 0.5, -3.3),
            (1.1, 0.5, -3.39),
            (0.616, 0.5, -2.51),
            (0.616, -0.5, -2.51),
            (1.1, -0.5, -3.39),
            (0.0148, 0.798, -3.9),
            (0.0148, 0.798, -2.3),
            (0.0148, -0.798, -2.3),
            (0.0148, -0.798, -3.9),
            (0.0148, 0.323, -2.79),
            (0.0148, -0.323, -2.79),
            (0.0148, -0.323, -3.44),
            (0.0148, 0.323, -3.44),
        ],
    "edges" :
        [
            (2, 0),
            (0, 1),
            (1, 3),
            (3, 2),
            (6, 2),
            (3, 7),
            (7, 6),
            (4, 6),
            (7, 5),
            (5, 4),
            (0, 4),
            (5, 1),
            (8, 9),
            (9, 10),
            (10, 11),
            (11, 8),
            (2, 8),
            (11, 3),
            (1, 10),
            (9, 0),
            (12, 13),
            (14, 13),
            (12, 15),
            (15, 14),
            (5, 15),
            (12, 7),
            (13, 6),
            (4, 14),
            (16, 17),
            (17, 18),
            (18, 19),
            (19, 16),
            (11, 19),
            (18, 10),
            (9, 17),
            (16, 8),
            (20, 21),
            (22, 21),
            (20, 23),
            (23, 22),
            (14, 22),
            (23, 15),
            (13, 21),
            (20, 12),
            (24, 25),
            (26, 25),
            (24, 27),
            (27, 26),
            (23, 27),
            (24, 20),
            (25, 21),
            (22, 26),
            (28, 29),
            (29, 30),
            (30, 31),
            (31, 28),
            (19, 31),
            (30, 18),
            (17, 29),
            (28, 16),
        ],
    "faces" :
        [
            (3, 2, 8, 11),
            (2, 3, 7, 6),
            (7, 5, 15, 12),
            (4, 5, 1, 0),
            (2, 6, 4, 0),
            (7, 3, 1, 5),
            (10, 11, 19, 18),
            (0, 1, 10, 9),
            (1, 3, 11, 10),
            (2, 0, 9, 8),
            (15, 14, 22, 23),
            (6, 7, 12, 13),
            (5, 4, 14, 15),
            (4, 6, 13, 14),
            (18, 19, 31, 30),
            (8, 9, 17, 16),
            (11, 8, 16, 19),
            (9, 10, 18, 17),
            (20, 23, 27, 24),
            (14, 13, 21, 22),
            (12, 15, 23, 20),
            (13, 12, 20, 21),
            [(25, 24, 27, 26), (29, 30, 31, 28)],
            (21, 20, 24, 25),
            (23, 22, 26, 27),
            (22, 21, 25, 26),
            (16, 17, 29, 28),
            (19, 16, 28, 31),
            (17, 18, 30, 29),
        ],
    "genus" : 1,
    "nr_holes" : 1,
  }


In [ ]:
%%rman --return-var=cuboboros_img --post-process="show_poly_info(cuboboros, cuboboros_img[0])"
%rib
%autodisplay

Format 400 400 1

Projection "perspective" "fov" [45]
Translate 0.5 2 5
LightSource "distantlight" 1

WorldBegin
Rotate -45 0 1 0
Rotate -120 1 0 0
%insval "render_poly(cuboboros)"
WorldEnd

In [ ]:
tetratetrahedron = \
  {
    "verts" :
        [
            (-0.184, -0.111, 1.92),
            (1.7, -0.111, -0.742),
            (-1.13, 1.52, -0.742),
            (-1.13, -1.74, -0.742),
            (-0.184, -0.111, -2.08),
            (0.759, -0.111, -0.742),
            (-0.655, 0.706, -0.742),
            (-0.655, -0.927, -0.742),
            (-0.0264, -0.383, 1.04),
            (0.916, -0.383, -0.298),
            (0.759, -1.74, 0.591),
            (-0.498, -1.2, -0.298),
            (-0.0264, 0.162, 1.04),
            (0.916, 0.162, -0.298),
            (-0.498, 0.978, -0.298),
            (0.759, 1.52, 0.591),
            (-0.493, -0.111, 1.05),
            (-2.06, -0.111, 0.604),
            (-0.965, 0.706, -0.285),
            (-0.965, -0.927, -0.285),
        ],
    "edges" :
        [
            (0, 3),
            (1, 3),
            (2, 3),
            (1, 2),
            (0, 1),
            (0, 2),
            (4, 7),
            (5, 7),
            (6, 7),
            (5, 6),
            (4, 5),
            (4, 6),
            (8, 11),
            (9, 11),
            (10, 11),
            (9, 10),
            (8, 9),
            (8, 10),
            (12, 15),
            (13, 15),
            (14, 15),
            (13, 14),
            (12, 13),
            (12, 14),
            (16, 19),
            (17, 19),
            (18, 19),
            (17, 18),
            (16, 17),
            (16, 18),
        ],
    "faces" :
        [
            [(0, 1, 2), (12, 14, 13)],
            [(0, 2, 3), (16, 19, 18)],
            [(0, 3, 1), (8, 9, 11),],
            [(1, 3, 2), (5, 6, 7)],
            (4, 6, 5),
            (4, 7, 6),
            (4, 5, 7),
            # (5, 6, 7),
            (8, 10, 9),
            (8, 11, 10),
            # (8, 9, 11),
            (9, 10, 11),
            # (12, 14, 13),
            (12, 15, 14),
            (12, 13, 15),
            (13, 14, 15),
            (16, 18, 17),
            # (16, 19, 18),
            (16, 17, 19),
            (17, 18, 19),
        ],
    "nr_holes" : 4,
  }


In [ ]:
%%rman --return-var=tetratetrahedron_img --post-process="show_poly_info(tetratetrahedron, tetratetrahedron_img[0])"
%rib
%autodisplay

Format 400 400 1

Projection "perspective" "fov" [45]
Translate 0 0 6
LightSource "distantlight" 1

WorldBegin
Rotate -30 0 1 0
Rotate 90 1 0 0
%insval "render_poly(tetratetrahedron)"
WorldEnd

In [ ]:
cube_frame_verts = \
    [
        (-0.5, -1, -0.5),
        (-0.5, -0.5, 1),
        (-0.5, 1, -0.5),
        (-0.5, 0.5, 1),
        (0.5, -1, -0.5),
        (0.5, -0.5, 1),
        (1, 0.5, -0.5),
        (0.5, 0.5, 1),
        (-0.5, -1, 0.5),
        (-0.5, 1, 0.5),
        (1, 0.5, 0.5),
        (0.5, -1, 0.5),
        (-0.5, -0.5, -1),
        (-1, -0.5, 0.5),
        (1, -0.5, 0.5),
        (1, -0.5, -0.5),
        (0.5, 1, 0.5),
        (0.5, 1, -0.5),
        (0.5, -0.5, -1),
        (1, -1, 1),
        (-1, 0.5, 0.5),
        (-0.5, 0.5, -1),
        (0.5, 0.5, -1),
        (1, 1, 1),
        (-1, 0.5, -0.5),
        (-1, -0.5, -0.5),
        (1, 1, -1),
        (1, -1, -1),
        (-1, 1, -1),
        (-1, -1, -1),
        (-1, -1, 1),
        (-1, 1, 1),
        (0.5, 0.5, 0.5),
        (-0.5, 0.5, 0.5),
        (0.5, -0.5, 0.5),
        (-0.5, -0.5, 0.5),
        (-0.5, -0.5, -0.5),
        (0.5, -0.5, -0.5),
        (0.5, 0.5, -0.5),
        (-0.5, 0.5, -0.5),
    ]
cube_frame = \
  {
    "verts" : cube_frame_verts,
    "edges" :
        [
            (28, 29),
            (29, 30),
            (30, 31),
            (31, 28),
            (26, 28),
            (31, 23),
            (23, 26),
            (27, 26),
            (23, 19),
            (19, 27),
            (29, 27),
            (19, 30),
            (1, 3),
            (3, 7),
            (7, 5),
            (5, 1),
            (30, 1),
            (3, 31),
            (7, 23),
            (5, 19),
            (0, 8),
            (11, 4),
            (0, 4),
            (11, 8),
            (29, 0),
            (8, 30),
            (19, 11),
            (4, 27),
            (10, 6),
            (15, 6),
            (10, 14),
            (14, 15),
            (23, 10),
            (6, 26),
            (15, 27),
            (19, 14),
            (9, 2),
            (17, 2),
            (9, 16),
            (16, 17),
            (31, 9),
            (2, 28),
            (17, 26),
            (23, 16),
            (21, 12),
            (22, 21),
            (18, 22),
            (12, 18),
            (28, 21),
            (12, 29),
            (26, 22),
            (27, 18),
            (24, 25),
            (25, 13),
            (13, 20),
            (20, 24),
            (29, 25),
            (24, 28),
            (30, 13),
            (31, 20),
            (20, 33),
            (10, 32),
            (14, 34),
            (6, 38),
            (13, 35),
            (22, 38),
            (17, 38),
            (32, 38),
            (37, 15),
            (37, 38),
            (34, 37),
            (32, 34),
            (11, 34),
            (4, 37),
            (36, 0),
            (36, 37),
            (35, 8),
            (35, 36),
            (34, 35),
            (33, 35),
            (25, 36),
            (39, 24),
            (39, 36),
            (33, 39),
            (3, 33),
            (33, 32),
            (32, 7),
            (5, 34),
            (1, 35),
            (18, 37),
            (39, 2),
            (39, 38),
            (16, 32),
            (21, 39),
            (12, 36),
            (9, 33),
        ],
    "faces" :
        [
            (1, 3, 31, 30),
            (3, 7, 23, 31),
            (7, 5, 19, 23),
            (5, 1, 30, 19),
            (0, 8, 30, 29),
            (11, 4, 27, 19),
            (4, 0, 29, 27),
            (8, 11, 19, 30),
            (10, 6, 26, 23),
            (6, 15, 27, 26),
            (14, 10, 23, 19),
            (15, 14, 19, 27),
            (9, 2, 28, 31),
            (2, 17, 26, 28),
            (16, 9, 31, 23),
            (17, 16, 23, 26),
            (21, 12, 29, 28),
            (22, 21, 28, 26),
            (18, 22, 26, 27),
            (12, 18, 27, 29),
            (25, 24, 28, 29),
            (13, 25, 29, 30),
            (20, 13, 30, 31),
            (24, 20, 31, 28),
            (38, 6, 10, 32),
            (38, 37, 15, 6),
            (37, 34, 14, 15),
            (14, 34, 32, 10),
            (4, 11, 34, 37),
            (37, 36, 0, 4),
            (0, 36, 35, 8),
            (8, 35, 34, 11),
            (33, 35, 13, 20),
            (25, 13, 35, 36),
            (36, 39, 24, 25),
            (24, 39, 33, 20),
            (7, 3, 33, 32),
            (5, 7, 32, 34),
            (1, 5, 34, 35),
            (1, 35, 33, 3),
            (22, 18, 37, 38),
            (17, 2, 39, 38),
            (17, 38, 32, 16),
            (22, 38, 39, 21),
            (12, 21, 39, 36),
            (18, 12, 36, 37),
            (2, 9, 33, 39),
            (16, 32, 33, 9),
        ],
    "genus" : 5,
    "nr_shells" : 1,
    "nr_holes" : 0,
  }


In [ ]:
%%rman --return-var=cube_frame_img --post-process="show_poly_info(cube_frame, cube_frame_img[0])"
%rib
%autodisplay

Format 400 400 1

Projection "perspective" "fov" [45]
Translate 0 0 5
LightSource "distantlight" 1

WorldBegin
Rotate 45 0 1 0
Rotate 45 1 0 0
%insval "render_poly(cube_frame)"
WorldEnd

You may need some convincing over that last one. Why is the genus set to 5, when there are holes in all 6 faces? Let’s take it step by step. The first step is to take a cube and cut a hole right through it. This produces a shape with a genus of 1, even though the hole penetrates 2 faces:

In [ ]:
cube_frame_1 = \
  {
    "verts" : cube_frame_verts,
    "edges" :
        [
            (28, 29),
            (29, 30),
            (30, 31),
            (31, 28),
            (26, 28),
            (31, 23),
            (23, 26),
            (27, 26),
            (23, 19),
            (19, 27),
            (29, 27),
            (19, 30),
            (1, 3),
            (3, 7),
            (7, 5),
            (5, 1),
            (30, 1),
            (3, 31),
            (7, 23),
            (5, 19),
            (0, 8),
            (11, 4),
            (0, 4),
            (11, 8),
            (29, 0),
            (8, 30),
            (19, 11),
            (4, 27),
            (10, 6),
            (15, 6),
            (10, 14),
            (14, 15),
            (23, 10),
            (6, 26),
            (15, 27),
            (19, 14),
            (9, 2),
            (17, 2),
            (9, 16),
            (16, 17),
            (31, 9),
            (2, 28),
            (17, 26),
            (23, 16),
            (21, 12),
            (22, 21),
            (18, 22),
            (12, 18),
            (28, 21),
            (12, 29),
            (26, 22),
            (27, 18),
            (24, 25),
            (25, 13),
            (13, 20),
            (20, 24),
            (29, 25),
            (24, 28),
            (30, 13),
            (31, 20),
            (20, 33),
            (10, 32),
            (14, 34),
            (6, 38),
            (13, 35),
            (32, 38),
            (37, 15),
            (37, 38),
            (34, 37),
            (32, 34),
            (36, 37),
            (35, 36),
            (34, 35),
            (33, 35),
            (25, 36),
            (39, 24),
            (39, 36),
            (33, 39),
            (33, 32),
            (39, 38),
        ],
    "faces" :
        [
            (1, 3, 31, 30),
            (3, 7, 23, 31),
            (7, 5, 19, 23),
            (5, 1, 30, 19),
            (0, 8, 30, 29),
            (11, 4, 27, 19),
            (4, 0, 29, 27),
            (8, 11, 19, 30),
            (10, 6, 26, 23),
            (6, 15, 27, 26),
            (14, 10, 23, 19),
            (15, 14, 19, 27),
            (9, 2, 28, 31),
            (2, 17, 26, 28),
            (16, 9, 31, 23),
            (17, 16, 23, 26),
            (21, 12, 29, 28),
            (22, 21, 28, 26),
            (18, 22, 26, 27),
            (12, 18, 27, 29),
            (25, 24, 28, 29),
            (13, 25, 29, 30),
            (20, 13, 30, 31),
            (24, 20, 31, 28),
            (38, 6, 10, 32),
            (38, 37, 15, 6),
            (37, 34, 14, 15),
            (14, 34, 32, 10),
            (33, 35, 13, 20),
            (25, 13, 35, 36),
            (36, 39, 24, 25),
            (24, 39, 33, 20),
            (37, 38, 39, 36),
            (37, 36, 35, 34),
            (0, 4, 11, 8),
            (21, 22, 18, 12),
            (34, 35, 33, 32),
            (1, 5, 7, 3),
            (2, 9, 16, 17),
            (38, 32, 33, 39),
        ],
    "genus" : 1,
  }


In [ ]:
%%rman
%rib
%autodisplay

Format 400 400 1

Projection "perspective" "fov" [30]
Translate -0.2 0.5 7
LightSource "distantlight" 1

WorldBegin
%insval "set_colour(mug_colour)"
%insval "set_opacity(mug_colour.a)"
TransformBegin
Rotate 60 1 0 0
Rotate 20 0 0 1
%insval "render_mesh(cube_frame_1, False)"
TransformEnd
WorldEnd

We then cut a hole through a third side, increasing the genus by one to 2:

In [ ]:
cube_frame_2 = \
  {
    "verts" : cube_frame_verts,
    "edges" :
        [
            (28, 29),
            (29, 30),
            (30, 31),
            (31, 28),
            (26, 28),
            (31, 23),
            (23, 26),
            (27, 26),
            (23, 19),
            (19, 27),
            (29, 27),
            (19, 30),
            (1, 3),
            (3, 7),
            (7, 5),
            (5, 1),
            (30, 1),
            (3, 31),
            (7, 23),
            (5, 19),
            (0, 8),
            (11, 4),
            (0, 4),
            (11, 8),
            (29, 0),
            (8, 30),
            (19, 11),
            (4, 27),
            (10, 6),
            (15, 6),
            (10, 14),
            (14, 15),
            (23, 10),
            (6, 26),
            (15, 27),
            (19, 14),
            (9, 2),
            (17, 2),
            (9, 16),
            (16, 17),
            (31, 9),
            (2, 28),
            (17, 26),
            (23, 16),
            (21, 12),
            (22, 21),
            (18, 22),
            (12, 18),
            (28, 21),
            (12, 29),
            (26, 22),
            (27, 18),
            (24, 25),
            (25, 13),
            (13, 20),
            (20, 24),
            (29, 25),
            (24, 28),
            (30, 13),
            (31, 20),
            (20, 33),
            (10, 32),
            (14, 34),
            (6, 38),
            (13, 35),
            (32, 38),
            (37, 15),
            (37, 38),
            (34, 37),
            (32, 34),
            (11, 34),
            (4, 37),
            (36, 0),
            (36, 37),
            (35, 8),
            (35, 36),
            (34, 35),
            (33, 35),
            (25, 36),
            (39, 24),
            (39, 36),
            (33, 39),
            (33, 32),
            (39, 38),
        ],
    "faces" :
        [
            (1, 3, 31, 30),
            (3, 7, 23, 31),
            (7, 5, 19, 23),
            (5, 1, 30, 19),
            (0, 8, 30, 29),
            (11, 4, 27, 19),
            (4, 0, 29, 27),
            (8, 11, 19, 30),
            (10, 6, 26, 23),
            (6, 15, 27, 26),
            (14, 10, 23, 19),
            (15, 14, 19, 27),
            (9, 2, 28, 31),
            (2, 17, 26, 28),
            (16, 9, 31, 23),
            (17, 16, 23, 26),
            (21, 12, 29, 28),
            (22, 21, 28, 26),
            (18, 22, 26, 27),
            (12, 18, 27, 29),
            (25, 24, 28, 29),
            (13, 25, 29, 30),
            (20, 13, 30, 31),
            (24, 20, 31, 28),
            (38, 6, 10, 32),
            (38, 37, 15, 6),
            (37, 34, 14, 15),
            (14, 34, 32, 10),
            (4, 11, 34, 37),
            (37, 36, 0, 4),
            (0, 36, 35, 8),
            (8, 35, 34, 11),
            (33, 35, 13, 20),
            (25, 13, 35, 36),
            (36, 39, 24, 25),
            (24, 39, 33, 20),
            (37, 38, 39, 36),
            (21, 22, 18, 12),
            (34, 35, 33, 32),
            (1, 5, 7, 3),
            (2, 9, 16, 17),
            (38, 32, 33, 39),
        ],
    "genus" : 2,
  }


In [ ]:
%%rman
%rib
%autodisplay

Format 400 400 1

Projection "perspective" "fov" [30]
Translate -0.2 0.5 7
LightSource "distantlight" 1

WorldBegin
%insval "set_colour(mug_colour)"
%insval "set_opacity(mug_colour.a)"
TransformBegin
Rotate 60 1 0 0
Rotate 20 0 0 1
%insval "render_mesh(cube_frame_2, False)"
TransformEnd
WorldEnd

Another hole through the opposite of the previous face, taking the genus to 3:

In [ ]:
cube_frame_3 = \
  {
    "verts" : cube_frame_verts,
    "edges" :
        [
            (28, 29),
            (29, 30),
            (30, 31),
            (31, 28),
            (26, 28),
            (31, 23),
            (23, 26),
            (27, 26),
            (23, 19),
            (19, 27),
            (29, 27),
            (19, 30),
            (1, 3),
            (3, 7),
            (7, 5),
            (5, 1),
            (30, 1),
            (3, 31),
            (7, 23),
            (5, 19),
            (0, 8),
            (11, 4),
            (0, 4),
            (11, 8),
            (29, 0),
            (8, 30),
            (19, 11),
            (4, 27),
            (10, 6),
            (15, 6),
            (10, 14),
            (14, 15),
            (23, 10),
            (6, 26),
            (15, 27),
            (19, 14),
            (9, 2),
            (17, 2),
            (9, 16),
            (16, 17),
            (31, 9),
            (2, 28),
            (17, 26),
            (23, 16),
            (21, 12),
            (22, 21),
            (18, 22),
            (12, 18),
            (28, 21),
            (12, 29),
            (26, 22),
            (27, 18),
            (24, 25),
            (25, 13),
            (13, 20),
            (20, 24),
            (29, 25),
            (24, 28),
            (30, 13),
            (31, 20),
            (20, 33),
            (10, 32),
            (14, 34),
            (6, 38),
            (13, 35),
            (17, 38),
            (32, 38),
            (37, 15),
            (37, 38),
            (34, 37),
            (32, 34),
            (11, 34),
            (4, 37),
            (36, 0),
            (36, 37),
            (35, 8),
            (35, 36),
            (34, 35),
            (33, 35),
            (25, 36),
            (39, 24),
            (39, 36),
            (33, 39),
            (33, 32),
            (39, 2),
            (39, 38),
            (16, 32),
            (9, 33),
        ],
    "faces" :
        [
            (1, 3, 31, 30),
            (3, 7, 23, 31),
            (7, 5, 19, 23),
            (5, 1, 30, 19),
            (0, 8, 30, 29),
            (11, 4, 27, 19),
            (4, 0, 29, 27),
            (8, 11, 19, 30),
            (10, 6, 26, 23),
            (6, 15, 27, 26),
            (14, 10, 23, 19),
            (15, 14, 19, 27),
            (9, 2, 28, 31),
            (2, 17, 26, 28),
            (16, 9, 31, 23),
            (17, 16, 23, 26),
            (21, 12, 29, 28),
            (22, 21, 28, 26),
            (18, 22, 26, 27),
            (12, 18, 27, 29),
            (25, 24, 28, 29),
            (13, 25, 29, 30),
            (20, 13, 30, 31),
            (24, 20, 31, 28),
            (38, 6, 10, 32),
            (38, 37, 15, 6),
            (37, 34, 14, 15),
            (14, 34, 32, 10),
            (4, 11, 34, 37),
            (37, 36, 0, 4),
            (0, 36, 35, 8),
            (8, 35, 34, 11),
            (33, 35, 13, 20),
            (25, 13, 35, 36),
            (36, 39, 24, 25),
            (24, 39, 33, 20),
            (37, 38, 39, 36),
            (17, 2, 39, 38),
            (17, 38, 32, 16),
            (21, 22, 18, 12),
            (34, 35, 33, 32),
            (1, 5, 7, 3),
            (2, 9, 33, 39),
            (16, 32, 33, 9),
        ],
    "genus" : 3,
  }


In [ ]:
%%rman
%rib
%autodisplay

Format 400 400 1

Projection "perspective" "fov" [30]
Translate -0.2 0.5 7
LightSource "distantlight" 1

WorldBegin
%insval "set_colour(mug_colour)"
%insval "set_opacity(mug_colour.a)"
TransformBegin
Rotate 60 1 0 0
Rotate 20 0 0 1
%insval "render_mesh(cube_frame_3, False)"
TransformEnd
WorldEnd

Now through the upper face, taking the genus to 4:

In [ ]:
cube_frame_4 = \
  {
    "verts" : cube_frame_verts,
    "edges" :
        [
            (28, 29),
            (29, 30),
            (30, 31),
            (31, 28),
            (26, 28),
            (31, 23),
            (23, 26),
            (27, 26),
            (23, 19),
            (19, 27),
            (29, 27),
            (19, 30),
            (1, 3),
            (3, 7),
            (7, 5),
            (5, 1),
            (30, 1),
            (3, 31),
            (7, 23),
            (5, 19),
            (0, 8),
            (11, 4),
            (0, 4),
            (11, 8),
            (29, 0),
            (8, 30),
            (19, 11),
            (4, 27),
            (10, 6),
            (15, 6),
            (10, 14),
            (14, 15),
            (23, 10),
            (6, 26),
            (15, 27),
            (19, 14),
            (9, 2),
            (17, 2),
            (9, 16),
            (16, 17),
            (31, 9),
            (2, 28),
            (17, 26),
            (23, 16),
            (21, 12),
            (22, 21),
            (18, 22),
            (12, 18),
            (28, 21),
            (12, 29),
            (26, 22),
            (27, 18),
            (24, 25),
            (25, 13),
            (13, 20),
            (20, 24),
            (29, 25),
            (24, 28),
            (30, 13),
            (31, 20),
            (20, 33),
            (10, 32),
            (14, 34),
            (6, 38),
            (13, 35),
            (17, 38),
            (32, 38),
            (37, 15),
            (37, 38),
            (34, 37),
            (32, 34),
            (11, 34),
            (4, 37),
            (36, 0),
            (36, 37),
            (35, 8),
            (35, 36),
            (34, 35),
            (33, 35),
            (25, 36),
            (39, 24),
            (39, 36),
            (33, 39),
            (33, 32),
            (39, 2),
            (39, 38),
            (16, 32),
            (9, 33),
            (3, 33),
            (32, 7),
            (34, 5),
            (35, 1),
        ],
    "faces" :
        [
            (1, 3, 31, 30),
            (3, 7, 23, 31),
            (7, 5, 19, 23),
            (5, 1, 30, 19),
            (0, 8, 30, 29),
            (11, 4, 27, 19),
            (4, 0, 29, 27),
            (8, 11, 19, 30),
            (10, 6, 26, 23),
            (6, 15, 27, 26),
            (14, 10, 23, 19),
            (15, 14, 19, 27),
            (9, 2, 28, 31),
            (2, 17, 26, 28),
            (16, 9, 31, 23),
            (17, 16, 23, 26),
            (21, 12, 29, 28),
            (22, 21, 28, 26),
            (18, 22, 26, 27),
            (12, 18, 27, 29),
            (25, 24, 28, 29),
            (13, 25, 29, 30),
            (20, 13, 30, 31),
            (24, 20, 31, 28),
            (38, 6, 10, 32),
            (38, 37, 15, 6),
            (37, 34, 14, 15),
            (14, 34, 32, 10),
            (4, 11, 34, 37),
            (37, 36, 0, 4),
            (0, 36, 35, 8),
            (8, 35, 34, 11),
            (33, 35, 13, 20),
            (25, 13, 35, 36),
            (36, 39, 24, 25),
            (24, 39, 33, 20),
            (37, 38, 39, 36),
            (17, 2, 39, 38),
            (17, 38, 32, 16),
            (21, 22, 18, 12),
            (2, 9, 33, 39),
            (16, 32, 33, 9),
            (3, 33, 32, 7),
            (7, 32, 34, 5),
            (5, 34, 35, 1),
            (1, 35, 33, 3),
        ],
    "genus" : 4,
  }


In [ ]:
%%rman
%rib
%autodisplay

Format 400 400 1

Projection "perspective" "fov" [30]
Translate 0.1 0.2 7
LightSource "distantlight" 1

WorldBegin
%insval "set_colour(mug_colour)"
%insval "set_opacity(mug_colour.a)"
TransformBegin
Rotate -120 1 0 0
Rotate 20 0 0 1
%insval "render_mesh(cube_frame_4, False)"
TransformEnd
WorldEnd

And you should be able to see at this point how the hole through the final face brings the genus to 5.

Well, the formula seems to hold up so far...